In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
!pip install category_encoders
import category_encoders as ce
from sklearn.metrics import accuracy_score

In [6]:
df=pd.read_csv('/content/fraudTrain.csv')
df_test=pd.read_csv('/content/fraudTest.csv')

##EDA

In [7]:
df.head(2)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0.0


In [8]:
df.shape

(875722, 23)

In [9]:
df.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,875722.000000,8.757220e+05,875722.000000,875722.000000,875722.000000,875722.000000,8.757220e+05,8.757220e+05,875722.000000,875721.000000,875721.000000
mean,437860.500000,4.167296e+17,70.288413,48795.477793,38.534339,-90.224501,8.901095e+04,1.342105e+09,38.534121,-90.224606,0.005670
std,252799.310555,1.308340e+18,162.749508,26900.808616,5.075532,13.766168,3.023269e+05,8.715097e+06,5.109892,13.778171,0.075083
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.325376e+09,19.027785,-166.671242,0.000000
25%,218930.250000,1.800400e+14,9.640000,26237.000000,34.668900,-96.798000,7.430000e+02,1.335010e+09,34.738022,-96.899015,0.000000
50%,437860.500000,3.520550e+15,47.450000,48174.000000,39.354300,-87.476900,2.456000e+03,1.342282e+09,39.364313,-87.435089,0.000000
75%,656790.750000,4.642255e+15,83.070000,72011.000000,41.894800,-80.158000,2.047800e+04,1.349552e+09,41.952545,-80.228024,0.000000
max,875721.000000,4.992346e+18,28948.900000,99783.000000,66.693300,-67.950300,2.906700e+06,1.356101e+09,67.510267,-66.956540,1.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875722 entries, 0 to 875721
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             875722 non-null  int64  
 1   trans_date_trans_time  875722 non-null  object 
 2   cc_num                 875722 non-null  int64  
 3   merchant               875722 non-null  object 
 4   category               875722 non-null  object 
 5   amt                    875722 non-null  float64
 6   first                  875722 non-null  object 
 7   last                   875722 non-null  object 
 8   gender                 875722 non-null  object 
 9   street                 875722 non-null  object 
 10  city                   875722 non-null  object 
 11  state                  875722 non-null  object 
 12  zip                    875722 non-null  int64  
 13  lat                    875722 non-null  float64
 14  long                   875722 non-nu

In [11]:
df.isnull().sum()


Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               1
is_fraud                 1
dtype: int64

In [12]:
df.dropna(inplace=True)

In [13]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

In [14]:
#Deleting unnecessary columns
df.drop(['Unnamed: 0', 'cc_num','first','last','unix_time','trans_num'],axis=1,inplace=True) #unix_time has wrong values
df_test.drop(['Unnamed: 0','cc_num','first','last','trans_num','unix_time'],axis=1,inplace=True)

In [15]:
# datetime formatting
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
# Convert datetime to Unix time (seconds since Unix epoch)
df['unix_time'] = df['trans_date_trans_time'].astype(int) // 10**9
df.drop('trans_date_trans_time',axis=1,inplace=True)

df_test['trans_date_trans_time'] = pd.to_datetime(df_test['trans_date_trans_time'])
df_test['unix_time'] = df_test['trans_date_trans_time'].astype(int) // 10**9
df_test.drop('trans_date_trans_time',axis=1,inplace=True)


In [16]:
df['dob'] = pd.to_datetime(df['dob'])

# Calculate age
current_date = pd.to_datetime('now')
age_timedelta = current_date - df['dob']
df['age'] = age_timedelta.dt.days / 365.25  # 365.25 days per year on average


df_test['dob'] = pd.to_datetime(df_test['dob'])

# Calculate age
current_date = pd.to_datetime('now')
age_timedelta = current_date - df_test['dob']
df_test['age'] = age_timedelta.dt.days / 365.25  # 365.25 days per year on average


In [17]:
df.drop('dob',axis=1,inplace=True)
df_test.drop('dob',axis=1,inplace=True)


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 875721 entries, 0 to 875720
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   merchant    875721 non-null  object 
 1   category    875721 non-null  object 
 2   amt         875721 non-null  float64
 3   gender      875721 non-null  object 
 4   street      875721 non-null  object 
 5   city        875721 non-null  object 
 6   state       875721 non-null  object 
 7   zip         875721 non-null  int64  
 8   lat         875721 non-null  float64
 9   long        875721 non-null  float64
 10  city_pop    875721 non-null  int64  
 11  job         875721 non-null  object 
 12  merch_lat   875721 non-null  float64
 13  merch_long  875721 non-null  float64
 14  is_fraud    875721 non-null  float64
 15  unix_time   875721 non-null  int64  
 16  age         875721 non-null  float64
dtypes: float64(7), int64(3), object(7)
memory usage: 120.3+ MB


In [19]:
gender_map = {'F': 1, 'M': 0}
df['gender'] = df['gender'].map(gender_map)
df_test['gender'] = df_test['gender'].map(gender_map)

##Modelling

In [21]:
X_train = df.drop(columns=['is_fraud'], axis=1)
y_train= df['is_fraud']
X_test= df_test.drop(columns=['is_fraud'], axis=1)
y_test= df_test['is_fraud']

In [22]:
# Define the target encoder with additive smoothing
encoder = ce.TargetEncoder(smoothing=0.1)

# Define the categorical columns to be encoded
categorical_columns = ['state', 'city', 'street', 'job', 'merchant', 'category']

# Fit and transform the categorical features
X_train_encoded = encoder.fit_transform(X_train[categorical_columns], y_train)
X_test_encoded = encoder.transform(X_test[categorical_columns],y_test)

# Replace the original categorical columns with the encoded versions
X_train.drop(columns=categorical_columns, inplace=True)
X_test.drop(columns=categorical_columns, inplace=True)
X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)



In [27]:
def print_score(y_true, y_pred, train=True):
    if train:
        print("Train Accuracy Score: {:.2f}%".format(accuracy_score(y_true, y_pred) * 100))
    else:
        print("Test Accuracy Score: {:.2f}%".format(accuracy_score(y_true, y_pred) * 100))

xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train, eval_metric='aucpr')

y_train_pred = xgb_clf.predict(X_train)
y_test_pred = xgb_clf.predict(X_test)

print_score(y_train, y_train_pred, train=True)
print_score(y_test, y_test_pred, train=False)



/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


Train Accuracy Score: 99.93%
Test Accuracy Score: 99.60%
